<div class="alert alert-block alert-success">
<h1><b>PROJET 9 - </b>Réalisez une application de recommandation de contenu</h1>
<h2>Notebook Scripts</h2></div>

<img src="my_content.png" width="250" height="250">

## Table of Contents<hr>
[INTRODUCTION & PRESENTATION](#section_0)<br>
[A - IMPORTATIONS](#section_0A)<br>
[B - DATA COLLECTION & DESCRIPTION - EDA](#section_0B)<br>
[C - PERFORMANCE METRICS](#section_0C)<br>
&emsp;[1 - Intersection over Union (IoU)](#section_0C1)<br>
&emsp;[2 - Dice Coefficient](#section_0C2)<br>
&emsp;[3 - Performance metrics calculation function](#section_0C3)<br>
&emsp;[4 - Loss function](#section_0C4)<br><hr>
[PART 1 - DATA PREPARATION](#section_1)<br>
[A - DATA AUGMENTATION](#section_1A)<br>
&emsp;[1 - Some imgaug techniques](#section_1A1)<br>
&emsp;[2 - imgaug augmentation pipeline](#section_1A2)<br>
&emsp;[3 - Images and masks augmentation](#section_1A3)<br>
[B - DATA GENERATOR](#section_1B)<br>
[C - FINAL DATASETS FOR MODELLING](#section_1C)<br><hr>
[PART 2 - CONVOLUTIONAL NEURAL NETWORKS MODELS](#section_2)<br>
&emsp;[1 - Presentation](#section_21)<br>
&emsp;[2 - Utils functions to build the U-NET models](#section_22)<br>
&emsp;[3 - Utils functions to visualize the training process](#section_23)<br>
[A - U-NET MINI (unet_mini) - BASE MODEL](#section_2A)<br><hr>
[CONCLUSION](#section_00)<br>

<h2><div class="alert alert-block alert-success" id="section_0">
INTRODUCTION & PRESENTATION</div></h2>

### Our mission

My Content est une start-up qui veut encourager la lecture en recommandant des contenus pertinents pour ses utilisateurs.

Vous êtes le CTO et cofondateur de la start-up avec Samia qui est CEO. Vous êtes en pleine construction d’un premier MVP qui prendra la forme d’une application.

Dans un premier temps, votre start-up souhaite tester une solution de recommandation d’articles et de livres à des particuliers.

"En tant qu’utilisateur de l’application, je vais recevoir une sélection de cinq articles."

### Origin of the datasets

Comme vous ne disposez pas à ce jour de données utilisateurs, vous allez utiliser des données disponibles en ligne pour développer votre MVP (en téléchargement direct à ce lien).

Ces données représentent les interactions des utilisateurs avec les articles disponibles. Elles contiennent des informations sur les articles (par exemple le nombre de mots dans l’article), ainsi que les informations sur les sessions des utilisateurs (par exemple heures de début et de fin) et les interactions des utilisateurs avec les articles (sur quel article l’utilisateur a-t-il cliqué lors de sa session ?).

une architecture serverless avec Azure Functions 

Dans la première architecture, tu crées une API pour développer puis exposer ton système de recommandation. Pour faire le lien entre l’application et le système de recommandation, tu crées une Azure Function

Dans la deuxième architecture, tu peux te passer de l’API, en exploitant les fonctionnalités “Azure Blob storage input binding” pour récupérer directement les fichiers et modèles, et en intégrant tes prédictions directement dans ton Azure Functions

Je te laisse choisir l’architecture que tu mettras en place !

Pour l’application, tu peux créer en local une simple interface qui liste les id des users et affiche les résultats des 5 suggestions d’articles, suite à appel de l’Azure Functions.

 fichier “embeddings” : tu peux réaliser une réduction de dimension de ce fichier via une ACP.

Il est conseillé de créer votre Azure Function directement sur le portail Azure, et de choisir le “plan type” égal à “Consomption(serverless)”, afin d’utiliser l’option gratuite de Service Plan

Sélectionner l’architecture logicielle permettant de répondre au besoin métier :<br> 
    • les différentes briques d’architecture nécessaires pour la réalisation de la solution répondant au besoin métier ont été identifiées.<br>
    • la description fonctionnelle de chaque brique d’architecture a été réalisée.<br>
    • les besoins de mise à jour des bases de données Utilisateurs et Articles ont été pris en compte dans la description fonctionnelle.<br>
    • une explication sous forme de schéma des différentes briques de l’architecture logicielle a été élaborée.<br>
    

Concevoir des scripts permettant d’exécuter une chaîne de traitements IA bout-en-bout :<br>
• les scripts permettant d’entraîner et de tester le modèle de recommandation ont été développés et testés.<br>
• au moins un des algorithmes de chacune des deux approches (CF et content-based) de système de recommandation a été testé.<br>
• la fonction serverless renvoyant la prédiction du modèle intégrée à l’application mobile a été testée.<br>
• les scripts sont découpés en fonctions et/ou classes permettant une meilleure lisibilité et une meilleure maintenabilité du code.<br>
• les scripts sont stockés dans un dossier GitHub.

<h3><div class="alert alert-block alert-info" id="section_0A">
A - IMPORTATIONS</div></h3>

In [1]:
%matplotlib inline

# Importation of Python modules and methods.
import os
import random
import warnings
from glob import glob

# Importation of Machine Learning libraries.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'  # Only warning and error information is displayed.

<h3><div class="alert alert-block alert-info" id="section_0B">
B - DATA COLLECTION & DESCRIPTION - EDA</div></h3>

In [2]:
# Creation of pandas DataFrames with 9 Olist .csv files.
df_clicks = pd.read_csv(
    'clicks_sample.csv', sep=',', low_memory=False)
df_articles = pd.read_csv(
    'articles_metadata.csv', sep=',', low_memory=False)

# Max display of rows and columns. 
pd.set_option('display.max_rows', 14000)
pd.set_option('display.max_columns', 120)

# Style for the plots.
plt.style.use('fivethirtyeight')

In [3]:
# Information and shape about the df_clicks DataFrame.
df_clicks.info(memory_usage='deep')
print('\033[1m' + f"\nThe dataset df_clicks has {df_clicks.shape[0]} \
observations and {df_clicks.shape[1]} variables." + '\033[0m')
# 5 samples of rows with all the columns.
df_clicks.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883 entries, 0 to 1882
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   user_id              1883 non-null   int64
 1   session_id           1883 non-null   int64
 2   session_start        1883 non-null   int64
 3   session_size         1883 non-null   int64
 4   click_article_id     1883 non-null   int64
 5   click_timestamp      1883 non-null   int64
 6   click_environment    1883 non-null   int64
 7   click_deviceGroup    1883 non-null   int64
 8   click_os             1883 non-null   int64
 9   click_country        1883 non-null   int64
 10  click_region         1883 non-null   int64
 11  click_referrer_type  1883 non-null   int64
dtypes: int64(12)
memory usage: 176.7 KB

The dataset df_clicks has 1883 observations and 12 variables.


,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
766,285,1506827353209022,1506827353000,2,66457,1506827753448,4,3,2,1,13,2
101,39,1506826064238776,1506826064000,2,284847,1506827891272,4,3,2,1,16,7
1829,686,1506828890351423,1506828890000,2,106886,1506828968287,4,3,20,1,24,1
1134,427,1506827857105164,1506827857000,4,108854,1506829153246,4,3,2,1,24,1
1341,512,1506828174351249,1506828174000,2,68866,1506828306154,4,3,20,1,21,1


In [4]:
# Number of missing values by column.
df_clicks.isna().sum()

user_id                0
session_id             0
session_start          0
session_size           0
click_article_id       0
click_timestamp        0
click_environment      0
click_deviceGroup      0
click_os               0
click_country          0
click_region           0
click_referrer_type    0
dtype: int64

In [5]:
print('\033[1m' + f"Number of missing values in the df_clicks dataframe: \
{df_clicks.isna().sum().sum()}" + '\033[0m')

Number of missing values in the df_clicks dataframe: 0


In [6]:
# Function to calculate the number of unique values for each columns of a dataframe.
def unique_values(df, col_list):
    """Function to generate the unique values of each column of a dataframe."""
    for col in col_list:
        col_unique_values = df[col].nunique()
        print(f"Number of unique {col}: {col_unique_values}")

In [7]:
# Number of unique values for each columns of df_clicks. 
cols = df_clicks.columns.tolist()
unique_values(df_clicks, cols)

Number of unique user_id: 707
Number of unique session_id: 707
Number of unique session_start: 619
Number of unique session_size: 11
Number of unique click_article_id: 323
Number of unique click_timestamp: 1883
Number of unique click_environment: 3
Number of unique click_deviceGroup: 3
Number of unique click_os: 6
Number of unique click_country: 7
Number of unique click_region: 26
Number of unique click_referrer_type: 6


In [8]:
# Information and shape about the df_articles DataFrame.
df_articles.info(memory_usage='deep')
print('\033[1m' + f"\nThe dataset df_articles has {df_articles.shape[0]} \
observations and {df_articles.shape[1]} variables." + '\033[0m')
# 5 samples of rows with all the columns.
df_articles.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364047 entries, 0 to 364046
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   article_id     364047 non-null  int64
 1   category_id    364047 non-null  int64
 2   created_at_ts  364047 non-null  int64
 3   publisher_id   364047 non-null  int64
 4   words_count    364047 non-null  int64
dtypes: int64(5)
memory usage: 13.9 MB

The dataset df_articles has 364047 observations and 5 variables.


,article_id,category_id,created_at_ts,publisher_id,words_count
102913,102913,228,1428359108000,0,155
222950,222950,354,1454013998000,0,232
361348,361348,455,1515599835000,0,154
111190,111190,231,1509784486000,0,184
280101,280101,412,1438198547000,0,142


In [9]:
# Number of missing values by column.
df_articles.isna().sum()

article_id       0
category_id      0
created_at_ts    0
publisher_id     0
words_count      0
dtype: int64

In [10]:
print('\033[1m' + f"Number of missing values in the df_articles dataframe: \
{df_articles.isna().sum().sum()}" + '\033[0m')

Number of missing values in the df_articles dataframe: 0


In [11]:
# Number of unique values for each columns of df_articles. 
cols = df_articles.columns.tolist()
unique_values(df_articles, cols)

Number of unique article_id: 364047
Number of unique category_id: 461
Number of unique created_at_ts: 359552
Number of unique publisher_id: 1
Number of unique words_count: 866


In [12]:
df_clicks_hour = pd.read_csv(
    'clicks/clicks_hour_000.csv', sep=',', low_memory=False)

In [13]:
# Information and shape about the df_clicks DataFrame.
df_clicks_hour.info(memory_usage='deep')
print('\033[1m' + f"\nThe dataset df_clicks_hour has {df_clicks_hour.shape[0]} \
observations and {df_clicks_hour.shape[1]} variables." + '\033[0m')
# 5 samples of rows with all the columns.
df_clicks_hour.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883 entries, 0 to 1882
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   user_id              1883 non-null   int64
 1   session_id           1883 non-null   int64
 2   session_start        1883 non-null   int64
 3   session_size         1883 non-null   int64
 4   click_article_id     1883 non-null   int64
 5   click_timestamp      1883 non-null   int64
 6   click_environment    1883 non-null   int64
 7   click_deviceGroup    1883 non-null   int64
 8   click_os             1883 non-null   int64
 9   click_country        1883 non-null   int64
 10  click_region         1883 non-null   int64
 11  click_referrer_type  1883 non-null   int64
dtypes: int64(12)
memory usage: 176.7 KB

The dataset df_clicks_hour has 1883 observations and 12 variables.


,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
761,283,1506827346241020,1506827346000,9,288984,1506829408845,4,1,17,1,14,1
616,232,1506827111416969,1506827111000,2,145166,1506827189281,4,3,2,1,25,2
1728,655,1506828772175392,1506828772000,4,161866,1506829519142,4,1,17,11,28,1
1843,692,1506828915900429,1506828915000,4,235840,1506829166634,4,3,19,1,25,2
1816,681,1506828869280418,1506828869000,24,270225,1506834907995,4,4,20,1,20,1


In [14]:
print('\033[1m' + f"Number of missing values in the df_clicks_hour dataframe: \
{df_clicks_hour.isna().sum().sum()}" + '\033[0m')

Number of missing values in the df_clicks_hour dataframe: 0


In [15]:
# Number of unique values for each columns of df_clicks_hour. 
cols = df_clicks_hour.columns.tolist()
unique_values(df_clicks_hour, cols)

Number of unique user_id: 707
Number of unique session_id: 707
Number of unique session_start: 619
Number of unique session_size: 11
Number of unique click_article_id: 323
Number of unique click_timestamp: 1883
Number of unique click_environment: 3
Number of unique click_deviceGroup: 3
Number of unique click_os: 6
Number of unique click_country: 7
Number of unique click_region: 26
Number of unique click_referrer_type: 6


In [16]:
# # Number of order(s) by 'customer_unique_id'.
# print('\033[1m' + "Number of order(s) by 'customer_unique_id':" +'\033[0m')
# df_customers.groupby("customer_unique_id").size().unique()

In [17]:
# # Top 20 of the city from where customers buy. 
# top_20_city_customers = df_customers.groupby(
#     ['customer_city'])['customer_unique_id'].count().sort_values(ascending=False)[:20]

# # Bar plotting the top 20 customers city.
# fig=plt.figure(figsize=(16,9))
# sns.barplot(y=top_20_city_customers.index, x=top_20_city_customers.values)
# plt.title('Top 20 City with most Customers', pad=15, fontsize=24)
# plt.xlabel('Number of customers', fontsize=20)
# plt.ylabel('City', fontsize=20)

<h3><div class="alert alert-block alert-info" id="section_0C">
C - PERFORMANCE METRICS</div></h3>

<b>To evaluate the performance of the Deep Learning models</b> we are going to build for <b>Semantic Segmentation</b>, we will use 2 different metrics, both quite similar.

Our prediction output shape, the mask, matches the input's spatial resolution (width and height) with a channel depth equivalent to the number of possible classes to be predicted. Each channel consists of a binary mask which labels areas where a specific class is present.

<b>Accuracy is here irrelevant</b>. Pixel accuracy can provide misleading results when the class representation is small within the image, as the measure will be biased in mainly reporting how well you identify negative case (ie. where the class is not present).

Instead, we will use here the <b>IoU and the Dice Coefficient</b> that suit more for semantic segmentation.

<h2><div class="alert alert-block alert-success" id="section_1">
PART 1 - DATA PREPARATION</div></h2>

<h3><div class="alert alert-block alert-info" id="section_1A">
A - DATA AUGMENTATION</div></h3>

<h2><div class="alert alert-block alert-success" id="section_00">
CONCLUSION</div></h2>

<img src="my_content.png" width="250" height="250">